# QEpy for bulk Al

## Nov.  06, 2023

pip install dftpy mpi4py mpi4py-fft qepy

In [2]:
from qepy.driver import Driver
from qepy.io import QEInput
from ase import Atoms
from ase.build import bulk
from ase.io import read

### Import needed DFTpy modules

In [3]:
import numpy as np

from dftpy.grid import DirectGrid
from dftpy.field import DirectField
from dftpy.functional import Functional, TotalFunctional, FunctionalOutput
from dftpy.functional.abstract_functional import AbstractFunctional
from dftpy.ions import Ions
from dftpy.constants import Units
from dftpy.formats import io
from dftpy.mpi import mp
from ase.io import read

In [4]:
import os
os.chdir('../..')
path_file = os.getcwd()

## MPI setup

In [5]:
try :
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    mp.comm = MPI.COMM_WORLD
except:
    ## Serial version also can be done without mpi4py
    comm = None

In [6]:
dictionary = {
    'Si_Btin': {'file': 'Si_Btin.vasp', 'structure': '', 'lattice':None,  'index': 9}, 
    'Si_fcc': {'file': 'Si_fcc.vasp', 'structure': 'fcc', 'lattice': 3.405,  'index': 16},
    'Si_bcc': {'file': 'Si_bcc.vasp', 'structure': 'bcc', 'lattice': None,  'index': 9},
    'Si_cd': {'file': 'Si_cd.vasp', 'structure': 'diamond', 'lattice':5.43,  'index': 10},
    'Si_dhcp': {'file': 'Si_dhcp.vasp', 'structure': 'dhcp', 'lattice':None,  'index': 10},
    'Si_hcp': {'file': 'Si_hcp.vasp', 'structure': 'hcp', 'lattice':None,  'index': 10},
    'Si_bct5': {'file': 'Si_bct5.vasp', 'structure': 'bct5', 'lattice':None,  'index': 10},
    'Si_sh': {'file': 'Si_sh.vasp', 'structure': 'sh', 'lattice':None,  'index': 9},
}

In [7]:
def get_ions(phase, r):
    if phase=='Si_fcc' or phase=='Si_8cd':
        print(dictionary[phase]['structure'])
        ions = Ions.from_ase(bulk('Si', str(dictionary[phase]['structure']), a= dictionary[phase]['lattice'], cubic=True))
#         print(l[i])
        cell = ions.get_cell()
        ions.set_cell(cell * r, scale_atoms=True) 
    if phase=='Si_dhcp':
        inputfile = path_file+'/Results/Structures/'+dictionary[phase]['file']
        ions = Ions.from_ase(read(inputfile, format='vasp'))
        cell = ions.get_cell()
        ions.set_cell(cell * r, scale_atoms=True) 
    else:
        inputfile = path_file+'/Results/Structures/'+dictionary[phase]['file']
        ions = Ions.from_ase(read(inputfile, format='vasp'))
        cell = ions.get_cell()
        ions.set_cell(cell * r, scale_atoms=True)
    return ions

In [8]:
r = np.linspace(0.8, 1.4, 30)

In [9]:
Phases = ['Si_dhcp', 'Si_hcp', 'Si_bct5']#, 'Si_sh'] 
atoms = []
for i, phase in enumerate(Phases):
    atoms.append(get_ions(phase, r[dictionary[phase]['index']]))

In [10]:
QE_options = []
for i, phase in enumerate(Phases):
    qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'/Users/valeria/Documents/PP/ofpp/EAC/upf/blps/'",
        'outdir': "'./tmp'",
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc' : 60,
        'occupations': "'smearing'"
    },
    '&electrons': {
        'conv_thr' : 1e-8,
    },
    'atomic_species': ['Si  28.08 si.lda.upf'],
#     'k_points automatic': ['11 11 11 0 0 0'],
    'k_points automatic': ['8 8 8 0 0 0'],
}
    qe_options = QEInput().update_atoms(atoms[i], qe_options)
    QE_options.append(qe_options)

### Run a full-SCF or directly read the converged density

In [11]:
# SCF
Rho_opt = []
ks_ke_opt = []
ks_te_opt = []
arr =[0]
for i, phase in enumerate(Phases):
    print(phase)
    driver=Driver(qe_options=QE_options[i], iterative = False, logfile='tmp.out', comm=comm)
    driver.scf()
    
    rho_f = driver.get_density()
    rho_opt = driver.data2field(rho_f)
    ions = Ions.from_ase(atoms[i])
    
    energy = driver.get_energy() / 2.0
    driver.calc_energy()
    ks_ke_opt.append(driver.embed.energies.ek)
    grid = driver.get_dftpy_grid(mp=mp)
    Rho_opt.append(rho_opt)
    ks_te_opt.append(energy)

Si_dhcp
Si_hcp
Si_bct5


In [12]:
Grid = []
for i, phase in enumerate(Phases):
    grid = Rho_opt[i].grid
    Grid.append(grid)
    if mp.is_root:
        print('Local:->', grid.nr, 'Global:->', grid.nrR, 'CPUs:->', mp.size, flush=True)

Local:-> [ 72  72 120] Global:-> [ 72  72 120] CPUs:-> 1
Local:-> [48 48 80] Global:-> [48 48 80] CPUs:-> 1
Local:-> [ 60  60 120] Global:-> [ 60  60 120] CPUs:-> 1


In [13]:
class ExternalPotential(AbstractFunctional):
    
    def __init__(self, potential, name='EXT', type='EXT', **kwargs):
        self.potential = potential
        self.name = name
        self.type = type
        
    def compute(self, density, **kwargs):
        energy = np.sum(density*self.potential)*density.grid.dV
        functional=FunctionalOutput(name=self.name, potential=self.potential, energy=energy)
        return functional

In [14]:
A = 0.01
q = np.zeros(3)
j_h=5
Vext=[]
V = []

for i, phase in enumerate(Phases):

    q[0] = 2*np.pi / Grid[i].cell.lengths()[0] # only x direction
    v = 2 * A * np.cos(np.einsum('i,ijkl->jkl', j_h*q, Grid[i].r))
    vext = ExternalPotential(v)
    Vext.append(vext)
    V.append(v)

### Total functional with vext

In [ ]:
Rho_f = []
Rho_diff = []
ks_ke_vext = []
ks_te_vext = []
for i, phase in enumerate(Phases[0:2]):
    print(i)
    qe_options = QE_options[i]
    qe_options['&electrons']['mixing_mode'] = "'TF'"
    qe_options['&system']['nosym'] = True
    if phase=='Si_hcp':
        qe_options['&system']['nr1'] = 25
        qe_options['&system']['nr2'] = 25
        qe_options['&system']['nr3'] = 48

    driver=Driver(qe_options=qe_options, iterative = False, logfile='tmp.out')
    rho_f = driver.field2data(Rho_opt[i])
    driver.set_density(rho_f)
    extpot_global = Grid[i].gather(V[i])
    print(extpot_global.shape)
    extpot = driver.field2data(extpot_global)*2.0 # Ha. to Ry
    driver.set_external_potential(potential=extpot, exttype=0)

    driver.electrons()
    energy = driver.calc_energy() / 2.0
    
    ks_ke_vext.append(driver.embed.energies.ek)
    print(energy)
    rho_f = driver.get_density()
    Rho_f.append(rho_f)
    if driver.is_root:
        rho_vext= driver.data2field(rho_f)
        rho_diff= rho_vext - Rho_opt[i]
#         rho_diff.write('rho_diff_8'+str(i)+'.xsf', ions=driver.get_dftpy_ions())
        rho2 = np.abs(rho_diff)
        ratio = rho_vext/Rho_opt[i]
    Rho_diff.append(rho2.integral()*0.5)
    ks_te_vext.append(energy)


0


In [ ]:
ke_diff = -np.asarray(ks_ke_opt)+np.asarray(ks_ke_vext)
te_diff = -np.asarray(ks_te_opt)+np.asarray(ks_te_vext)
rho_diff = np.abs(Rho_diff)

In [ ]:
print(ke_diff)
print(te_diff)
print(rho_diff)

In [ ]:
np.save('/Users/valeria/Documents/aiWT/Final_version/wt/wt/Results/Response/ks_ke_diff_j55', ke_diff)
np.save('/Users/valeria/Documents/aiWT/Final_version/wt/wt/Results/Response/ks_te_diff_j55', te_diff)
np.save('/Users/valeria/Documents/aiWT/Final_version/wt/wt/Results/Response/ks_rho_diff_j55', Rho_diff)

In [18]:
from dftpy.formats import io
# rho_diff_bct5_DEN = io.read_density('rho_diff_j1_DEN.xsf')
# rho_diff_bct5_ENE = io.read_density('rho_diff_j1_ENE.xsf')
# rho_diff_bct5_KEN = io.read_density('rho_diff_j1_KEN.xsf')
# rho_diff_bct5_AVE = io.read_density('rho_diff_j1_AVE.xsf')


In [20]:
rho_resp_diff_DEN = (rho_diff - rho_diff_bct5_DEN)
rho_resp_diff_ENE = (rho_diff - rho_diff_bct5_ENE)
rho_resp_diff_KEN = (rho_diff - rho_diff_bct5_KEN)
rho_resp_diff_AVE = (rho_diff - rho_diff_bct5_AVE)

In [21]:
rho_diff_bct5_DEN.write('rho_resp_diff_j5_DEN.xsf', ions=ions)
rho_diff_bct5_ENE.write('rho_resp_diff_j5_ENE.xsf', ions=ions)
rho_diff_bct5_KEN.write('rho_resp_diff_j5_KEN.xsf', ions=ions)
rho_diff_bct5_AVE.write('rho_resp_diff_j5_AVE.xsf', ions=ions)

In [19]:
rho_diff_bct5_DEN = io.read_density('rho_diff_j5_DEN.xsf')
rho_diff_bct5_ENE = io.read_density('rho_diff_j5_ENE.xsf')
rho_diff_bct5_KEN = io.read_density('rho_diff_j5_KEN.xsf')
rho_diff_bct5_AVE = io.read_density('rho_diff_j5_AVE.xsf')

In [30]:
rho_resp_diff = (rho_diff - rho_diff_bct5)

In [32]:
rho_resp_diff_.write('rho_resp_diff_j1_DEN.xsf', ions=ions)